In [1]:
import aoc
import numpy as np
import math
import re
from functools import reduce

# Day 3

In [3]:
raw_data = aoc.get_data(2020, 3)

In [4]:
data = raw_data.replace('.', '0').replace('#', '1').splitlines()

In [5]:
patch = np.array([[int(y) for y in x] for x in data])

In [8]:
slope = (3, 1)

In [9]:
def make_forest(patch, slope):
    
    shape = patch.shape
    
    max_down = shape[0] * slope[1]
    
    max_right = max_down * slope[0]
    
    return np.concatenate([patch] * math.ceil(max_right / shape[1]), axis=1)

In [10]:
forest = make_forest(patch, (1, 3))

In [11]:
forest.shape

(323, 992)

In [12]:
def count_trees(forest, slope):
    
    x = 0
    y = 0
    
    trees = []
    
    while True:
        try: 
            x += slope[0]
            y += slope[1]
        
            trees.append(forest[y, x])
        except IndexError:
            break
    
    return sum(trees)

In [13]:
count_trees(forest, slope)

187

# Day 4

In [14]:
raw_data = aoc.get_data(2020, 4)

In [15]:
passports = raw_data.split('\n\n')

In [16]:
test = """eyr:1972 cid:100
hcl:#18171d ecl:amb hgt:170 pid:186cm iyr:2018 byr:1926

iyr:2019
hcl:#602927 eyr:1967 hgt:170cm
ecl:grn pid:012533040 byr:1946

hcl:dab227 iyr:2012
ecl:brn hgt:182cm pid:021572410 eyr:2020 byr:1992 cid:277

hgt:59cm ecl:zzz
eyr:2038 hcl:74454a iyr:2023
pid:3556412378 byr:2007""".split('\n\n')

In [17]:
test

['eyr:1972 cid:100\nhcl:#18171d ecl:amb hgt:170 pid:186cm iyr:2018 byr:1926',
 'iyr:2019\nhcl:#602927 eyr:1967 hgt:170cm\necl:grn pid:012533040 byr:1946',
 'hcl:dab227 iyr:2012\necl:brn hgt:182cm pid:021572410 eyr:2020 byr:1992 cid:277',
 'hgt:59cm ecl:zzz\neyr:2038 hcl:74454a iyr:2023\npid:3556412378 byr:2007']

In [18]:
required_keys = {'byr', 'iyr', 
'eyr' ,
'hgt' ,
'hcl' ,'ecl', 
'pid' ,
}

In [19]:
required_keys

{'byr', 'ecl', 'eyr', 'hcl', 'hgt', 'iyr', 'pid'}

In [20]:
def validate_all_passport_fields(passport):
    
    items = passport.replace(' ', '\n').splitlines()
    
    document = dict()
    
    for item in items:
    
        key, value = item.split(':')
        
        document[key] = value
    
    if set(document.keys()).intersection(required_keys) == required_keys:
        
        return document
    
    
def valid_between(value, low, high):
    
    value = int(value)
    
    return low <= value <= high


def valid_height(value):

    if re.search('\d*in', value) is not None:
        
        return valid_between(value[:-2], 59, 76)
    
    elif re.search('\d*cm', value) is not None:
        
        return valid_between(value[:-2], 150, 193)
        
    else:
        return False

def valid_hair(value):

    return re.search('#[0-9a-f]{6}', value) is not None

def valid_eye(value):

    return value in {'amb', 'blu', 'brn', 'gry', 'grn', 'hzl', 'oth'}

def valid_passport_number(value):
    
    return re.search('^\d{9}$', value) is not None

    
def validate_passport(passport):
    
    document = validate_all_passport_fields(passport)
    
    if document:
        
        return all([
            valid_between(document['byr'], 1920, 2002),
            valid_between(document['iyr'], 2010, 2020),
            valid_between(document['eyr'], 2020, 2030),
            valid_height(document['hgt']),
            valid_hair(document['hcl']), 
            valid_eye(document['ecl']), 
            valid_passport_number(document['pid'])])
    else:
        return False
            
        

In [21]:
sum(map(lambda x: x is not None , map(validate_all_passport_fields, passports)))

237

In [22]:
sum(map(validate_passport, passports))

172

# Day 5

In [58]:
raw_data = aoc.get_data(2020, 5)

In [59]:
data = raw_data.splitlines()

In [77]:
def search(string, array, low):
    
    array_length = len(array)
    
    if array_length == 1:
        return array[0]
    
    comparitor = [*string].pop(0)
    
    if comparitor == low:
        
        return search(string[1:], array[: array_length//2], low)
    else:
        
        return search(string[1:], array[array_length//2:], low)

In [78]:
def get_seat(code):
    
    row_code = code[:-3]
    col_code = code[-3:]
    
    row = search(row_code, list(range(128)), 'F')
    col = search(col_code, list(range(8)), 'L')
    
    return row * 8 + col
    

In [79]:
seats = set(map(get_seat, data))

In [80]:
max(seats)

806

In [82]:
all_seats = set(range(min(seats), max(seats)))

In [83]:
all_seats.difference(seats)

{562}

# Day 6 

In [8]:
raw_data = aoc.get_data(2020, 6)

In [9]:
data = raw_data.split('\n\n')

In [10]:
def count_unique_questions(group):
    
    return len(set(group.replace('\n', '')))

In [11]:
sum(map(count_unique_questions, data))

6714

In [6]:
def count_common_questions(group):
    
    people = [set(x) for x in group.splitlines()]
    
    return len(reduce(lambda a,b: a.intersection(b) , people))

In [7]:
sum(map(count_common_questions, data))

3435

# Day 7

In [12]:
raw_data = aoc.get_data(2020,7)

In [16]:
data = raw_data.splitlines()

In [30]:
def make_bag_dict(bag_rules):
    
    bags = dict()
    
    for rule in bag_rules:
        
        color, contents = rule.replace('.', '').split(' bags contain ')
    
        contents = contents.split(', ')

        bags[color] = contents
        
    return bags


In [35]:
rules = make_bag_dict(data)
len(rules.keys())

594

In [34]:
target = 'shiny gold'

In [33]:
def find_target_bag(bag, target):
    
    contents = rules[bag]
    
    if len(contents) == 1 and target in contents:
        return True:
        
    return any()
    

{'vibrant bronze': ['3 dim olive bags'],
 'shiny teal': ['1 posh green bag',
  '5 pale indigo bags',
  '1 mirrored purple bag'],
 'striped aqua': ['5 bright orange bags'],
 'clear chartreuse': ['3 dotted black bags', '2 wavy olive bags'],
 'light lime': ['1 posh silver bag',
  '5 clear orange bags',
  '2 light olive bags',
  '3 dull maroon bags'],
 'light olive': ['4 striped turquoise bags'],
 'shiny purple': ['2 posh silver bags',
  '3 striped silver bags',
  '5 shiny beige bags',
  '2 plaid chartreuse bags'],
 'mirrored crimson': ['2 faded cyan bags'],
 'shiny turquoise': ['5 dull purple bags'],
 'dim red': ['2 dim salmon bags', '2 faded orange bags', '5 muted aqua bags'],
 'vibrant yellow': ['5 mirrored white bags',
  '5 vibrant blue bags',
  '3 mirrored lavender bags',
  '1 wavy cyan bag'],
 'posh salmon': ['1 dull black bag',
  '1 striped indigo bag',
  '1 muted silver bag',
  '2 vibrant crimson bags'],
 'pale black': ['1 plaid cyan bag'],
 'dotted salmon': ['3 wavy brown bags',
 